For the input of pyclone (https://github.com/Roth-Lab/pyclone) prepar a table that includes all the SNVs from the 2 timepoints the samples were collected and for each SNV set the normal copy number (normal_cn) to 2 (unless it is a variant on the Y chromosome then normal_cn is set to 1). For each variant check if it is located in regions of CNVs (using the CNV_FACETS output - https://github.com/dariober/cnv_facets#variants). So if the variant was where a CNV is located, the copy number of the minor allele in the malignant cells (minor_cn) and the copy number of the major allele in the malignant cells (major_cn) are set to the values "LCN_EM" and "TCN_EM - LCN_EM" from the CNV_facets output.

This script is run in a python 3.5 environment.

In [1]:
import vcf
import pandas as pd
import numpy as np

In [2]:
def pyclone_prep(T_sample_snvs, T_sample_cnvs, genotype):
    
    d = {'mutation_id' : [],
        'ref_counts' : [],
        'var_counts' : [],
        'normal_cn' : [],
        'minor_cn' : [], 
        'major_cn' : []
        }
    

    for record in T_sample_snvs:
        #print(record)
        chrom = record.CHROM
        pos = record.POS
        ref = record.REF
        alt = record.ALT[0]  ##because it has the form of a list
        rsid = record.ID
        AD_ref = record.genotype(genotype).data.AD[0]
        AD_alt = record.genotype(genotype).data.AD[1]
    
    
        if rsid == None:
            mutation_ID = str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt)
        else:
            mutation_ID = str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt) + ":" + str(rsid)
          
            
        if chrom == "chrY":
            normal_cn = 1
        else:
            normal_cn = 2

    
    
        T_sample_cnvs = T_sample_cnvs
        for record_ in T_sample_cnvs:
            chrom_ = record_.CHROM
            pos_ = record_.POS
            end_ = record_.INFO['END']
        
            tcn_ = record_.INFO['TCN_EM']
            minor_cn_ = record_.INFO['LCN_EM']
            if minor_cn_ == None:
                minor_cn_ = 0
            major_cn_ = tcn_ - minor_cn_
   
            if chrom == chrom_:
                if pos >= pos_ and pos <= end_ :
                    minor_cn = minor_cn_
                    major_cn = major_cn_
                else:
                    minor_cn = 1
                    major_cn = 1
                

        d['mutation_id'].append(mutation_ID)
        d['ref_counts'].append(AD_ref)
        d['var_counts'].append(AD_alt)
        d['normal_cn'].append(normal_cn)
        d['minor_cn'].append(minor_cn)
        d['major_cn'].append(major_cn)
        
    
    df = pd.DataFrame(d)
    
    return df

    

When the one of the input files is a maf file, use the following function.
This will be useful when it is needed to see the gene names for each SNV after pyclone has run.

In [ ]:
def pyclone_prep_maf(T_sample_snvs, T_sample_cnvs, genotype):
    
    d = {'mutation_id' : [],
        'ref_counts' : [],
        'var_counts' : [],
        'normal_cn' : [],
        'minor_cn' : [], 
        'major_cn' : []
        }
    
     
    # loop through the rows using iterrows()
    for index, row in T_sample_snvs.iterrows():
        gene = row['Hugo_Symbol']
        chrom = row['Chromosome']
        pos = row['Start_Position'] 
        ref = row['Reference_Allele'] 
        alt = row['Tumor_Seq_Allele2']
        rsid = row['dbSNP_RS']
        AD_ref = row['t_ref_count']
        AD_alt = row['t_alt_count']
    
    
        if rsid == None:
            mutation_ID = str(gene) + ":" + str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt)
        else:
            mutation_ID = str(gene) + ":" + str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt) + ":" + str(rsid)
          
            
        if chrom == "chrY":
            normal_cn = 1
        else:
            normal_cn = 2

    
    
        T_sample_cnvs = T_sample_cnvs
        for record_ in T_sample_cnvs:
            chrom_ = record_.CHROM
            pos_ = record_.POS
            end_ = record_.INFO['END']
        
            tcn_ = record_.INFO['TCN_EM']
            minor_cn_ = record_.INFO['LCN_EM']
            if minor_cn_ == None:
                minor_cn_ = 0
            major_cn_ = tcn_ - minor_cn_
   
            if chrom == chrom_:
                if pos >= pos_ and pos <= end_ :
                    minor_cn = minor_cn_
                    major_cn = major_cn_
                else:
                    minor_cn = 1
                    major_cn = 1
                

        d['mutation_id'].append(mutation_ID)
        d['ref_counts'].append(AD_ref)
        d['var_counts'].append(AD_alt)
        d['normal_cn'].append(normal_cn)
        d['minor_cn'].append(minor_cn)
        d['major_cn'].append(major_cn)
        
    
    df = pd.DataFrame(d)
    
    return df

    

Example of using the above function with inputs vcf files (after SNV and CNV calling):

In [ ]:
T1_sampleID_snvs = vcf.Reader(open('path/to/T1_sampleID.strelka_muTect.somatic.snv.vcf', 'r'))

T1_sampleID_cnvs = vcf.Reader(open('path/to/T1_sampleID.vcf', 'r'))


df = pyclone_prep(T_sample_snvs = T1_sampleID_snvs, T_sample_cnvs = T1_sampleID_cnvs, genotype = 'T1_sampleID')

df.to_csv('T1_sampleID.tsv', sep="\t", index = False)

In [ ]:
T2_sampleID_snvs = vcf.Reader(open('path/to/T2_sampleID.strelka_muTect.somatic.snv.vcf', 'r'))

T2_sampleID_cnvs = vcf.Reader(open('path/to/T2_sampleID.vcf', 'r'))


df = pyclone_prep(T_sample_snvs = T2_sampleID_snvs, T_sample_cnvs = T2_sampleID_cnvs, genotype = 'T2_sampleID')

df.to_csv('T2_sampleID.tsv', sep="\t", index = False)

Example using a MAF file to extract the SNVs

In [ ]:
T_sampleID_snvs = pd.read_csv('T_sampleID.strelka_muTect.somatic.snv.maf',
                              delimiter='\t',
                              skiprows=0,
                              header=1
                             )

T_sampleID_cnvs = vcf.Reader(open('T_sampleID.vcf', 'r'))


df = pyclone_prep_maf(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, genotype = 'sampleID')

df.to_csv('T_sampleID.tsv', sep="\t", index = False)